p()

In [1]:
import numpy as np

In [2]:
PATH_TO_DATA = "naivebayes-21\\trg.csv"

In [3]:
from collections import defaultdict

In [4]:
list_data = []
class_freq = defaultdict(lambda: 0)

with open(PATH_TO_DATA) as csv_file:
    for line in csv_file:
        line_lst = line.split(',')
        line_lst[-1] = line_lst[-1].replace('\n', '')

        class_freq[line_lst[1]] += 1 

        list_data.append(line_lst)


full_csv_data = np.array(list_data)
class_freq.pop('class')


labels = class_freq.keys()
headers = full_csv_data[0]
data = np.asarray(full_csv_data[1:,1:])
print(data[:2])

[['B'
  '"the 4 202 353 bp genome of the alkaliphilic bacterium bacillus halodurans c-125 contains 4066 predicted protein coding sequences cdss 2141 527 of which have functional assignments 1182 29 of which are conserved cdss with unknown function and 743 18 3 of which have no match to any protein database among the total cdss 88 match sequences of proteins found only in bacillus subtilis and 667 are widely conserved in comparison with the proteins of various organisms including bsubtilis the b halodurans genome contains 112 transposase genes indicating that transposases have played an important evolutionary role in horizontal gene transfer and also in internal genetic rearrangement in the genome strain c-125 lacks some of the necessary genes for competence such as coms srfa and rapc supporting the fact that competence has not been demonstrated experimentally in c-125 there is no paralog of tupa encoding teichuronopeptide which contributes to alkaliphily in the c-125 genome and an orth

In [5]:
print(class_freq.items())
print(labels)

dict_items([('B', 1602), ('A', 128), ('E', 2144), ('V', 126)])
dict_keys(['B', 'A', 'E', 'V'])


In [16]:
from numpy.random import default_rng

ABSTRT_I = 1
LBL_I = 0

In [7]:
def train_test_split(X, y, test_size=None, stratify=False):

    rng = default_rng()
    arr = np.arange(10)
    arr1 = np.arange(10)

    rng.random(arr)
    print(arr)

In [8]:
train_test_split(None, None, None)

[0 1 2 3 4 5 6 7 8 9]


In [17]:
 def get_stratified_kfold_splits(data, k=10):
    instances_of_class = dict()

    for label in labels:
        i_arr = data[:, 0] == label
        instances_of_class[label] = data[i_arr]
    
    stratified_splits = dict()
    
    # Upsample to divisible by k
    for class_label, class_instances in instances_of_class.items():
        n = len(class_instances)
        upsample_amt = k - n % k

        random_indices = np.random.choice(class_instances[:, ABSTRT_I], size=upsample_amt, replace=False)
        random_indices.resize(random_indices.shape[0], 2)
        random_indices[:, -1] = class_label

        upsampled_class_instances = np.concatenate((class_instances, random_indices), 0)
        assert len(upsampled_class_instances) % k == 0, "num examples should be divisible by k"

        stratified_splits[class_label] = np.split(upsampled_class_instances, k)

    for i in range(k):
        kth_data_lst = [stratified_splits[label][i] for label in labels]
        
        kth_data = np.concatenate(kth_data_lst)
        np.random.shuffle(kth_data)

        yield kth_data

In [40]:
stratified_data = get_stratified_kfold_splits(data)

In [55]:
def get_word_counts(data):

    all_words = set()

    freq = []

    for i in range(len(data)):
        word_freq = defaultdict(lambda: 0)
        freq.append(word_freq)
        
        for word in data[i][ABSTRT_I].split():
            all_words.add(word)
            word_freq[word] += 1

    freq_mat = np.zeros((len(data), len(all_words)))

    all_words_lst = list(all_words)

    for j, word in enumerate(all_words_lst):
        for i in range(len(data)):
            freq_mat[i, j] = freq[i][word]

    return freq_mat

In [56]:
freq_mat = get_word_counts(next(stratified_data))

402
8880
(402, 8880)


In [64]:
def pca(X, y):
    features = X.T
    cov_matrix = np.cov(features)

    values, vectors = np.linalg.eig(cov_matrix)
    return values

In [65]:
print(freq_mat.shape)
values = pca(freq_mat[:, :-1], freq_mat[:, -1])

(402, 8880)


In [66]:
print(values)

[ 6.09963145e+01+0.00000000e+00j  1.29625288e+01+0.00000000e+00j
  1.06840753e+01+0.00000000e+00j ...  4.84039481e-51-5.79353677e-51j
 -5.78923121e-51+0.00000000e+00j -9.68948010e-52+0.00000000e+00j]


In [69]:
explained_variances = []
print(len(values))
for i in range(len(values)):
    explained_variances.append(values[i] / np.sum(values))
 
print(np.sum(explained_variances), explained_variances)

8879
(0.9999999999999999+1.0580856984207706e-33j) [(0.18243832504289506+3.953598670804545e-34j), (0.03877057259646426+8.401923458115071e-35j), (0.03195577990455776+6.925098053025695e-35j), (0.030060344550927383+6.514340571397476e-35j), (0.021268989109546634+4.6091766657630086e-35j), (0.018395610941485434+3.986490390645402e-35j), (0.017270428319390716+3.742653438180502e-35j), (0.014530199680677668+3.148821835025341e-35j), (0.01389334806082767+3.0108104979257126e-35j), (0.011865296342974443+2.5713138858984874e-35j), (0.011435419183324556+2.478155731404281e-35j), (0.010241365019236087+2.2193937111490062e-35j), (0.009848306060645016+2.1342143840603532e-35j), (0.008895267661637675+1.927682596035266e-35j), (0.008684887015921669+1.8820912631248021e-35j), (0.008596287645110813+1.8628910016342647e-35j), (0.008404652340302876+1.8213619486685712e-35j), (0.007898338523592275+1.711639299532961e-35j), (0.007835446674075198+1.6980100835994833e-35j), (0.007093955884525361+1.537322519772706e-35j), (0.0